In [2]:
import os
import base64
import hashlib
import json
import sys
import urllib
import glob
import pydicom
import pydicom_seg
import tempfile
import shutil
import numpy as np
import pandas as pd
import SimpleITK as sitk 
import subprocess as sp
import pyorthanc
import httpx

from typing import Literal
from httpx import HTTPError
from datetime import datetime
from pyorthanc import Orthanc
from collections import defaultdict


In [3]:
from pyorthanc import Orthanc, Study, Series


In [4]:
orthanc_client = Orthanc('http://localhost:8042')
orthanc_client.setup_credentials('dev-user-alta', 'SyTP&8JbKFx@a6R65^sE`Z$') 

# test conntection
patient_ids = orthanc_client.get_patients()
len(patient_ids)

9489

In [5]:
zonal_cases = glob.glob("/data/oleksii/Prostate-ZONE-Datasets-NRRDS/ALTA-Zone-Dataset/*/Segmentation-label.seg.nrrd")
seg_oid = [i.split("/")[-2] for i in zonal_cases]

In [6]:
seg_oid

['7b410c99-7e1e862d-3706a964-6bdbe57d-d2aff0ee',
 '4537c253-fbe4854b-38cfb239-ae6d6335-0cb37cdb',
 '429a4e0c-cf55e0ac-4c946d03-8ae810cd-76b18564',
 '7eb033b2-daed2e31-63c336f5-64aa9a14-4809977b',
 'ed5dfad9-29150db0-29de3a58-917e883b-7d3fbc6a',
 '308e4910-7ef940de-eb5c01ff-d56e0b54-d0b57019',
 '247e4f11-7a9de86f-3bbdb45a-aa0bc54b-3acdb232',
 '43f370cf-93b0c377-81db9e7c-6f45f968-d1c572af',
 '6144633d-85b8371b-50df1b60-4d19a62e-f97d91f8',
 'e6da5fce-4a72b848-486e26fd-ba5a740c-d76ee929',
 'fa8e6261-f0507bc6-ed868634-0bf41e81-9325372f',
 '1768cf3d-9c3edb24-e318e408-0067a0c8-45ece19b',
 'bcc6674f-2fc2a372-8d5cfa2e-0eb76e48-f9495e2e',
 '6ce3ab6a-a67c5542-1af66c61-75a217df-d131cb59',
 'aa0587c6-7ef11a3f-1c8ab209-1dbf9f0f-cfbdab10',
 '4db9a843-91a14378-593d3c59-15c18558-92130130',
 '2f9b0bbc-adb55379-23f3a8a7-55566081-b3e68c70',
 'c5b1b8c8-8b40a50a-db69d1f6-b7b76d75-6e98da30',
 'f4d9f672-7961c737-6361dea3-fa35aa7c-b4019e6e',
 'fde50109-70824306-a0f8a1a1-e52de2cd-3b586fe6',
 '8a91381f-3caa2c93-

In [7]:
some_missing = []
records = [] 
for soid in seg_oid:
    d = {}

    try:
        series = Series(soid, orthanc_client)
        study = series.parent_study
        # study = Study(series.study_identifier, orthanc_client)
        main_info = study.get_main_information()
        study_date = main_info['MainDicomTags']['StudyDate']
        study_UID = main_info['MainDicomTags']['StudyInstanceUID']
        patient_id = main_info['PatientMainDicomTags']['PatientID']
        
        d['study_date'] = study_date
        d['patient_id'] = patient_id
        d['StudyInstanceUID'] = study_UID
        d['study_orthanc_id'] = study.id_
        d['t2w_series_oid'] = soid

        records.append(d)

    except httpx.HTTPError as e:
#         print(f'A HTTPError was thrown: {e} ')
        some_missing.append(soid)
    
metadata_zone = pd.DataFrame(records)

In [8]:
metadata_zone

,study_date,patient_id,StudyInstanceUID,study_orthanc_id,t2w_series_oid
0,20180705,14765,1.3.12.2.1107.5.2.19.145132.300000180705060928...,ff354ce1-14e76513-1658b07f-9bb8a393-f118ec03,7b410c99-7e1e862d-3706a964-6bdbe57d-d2aff0ee
1,20151103,2690680000,1.2.826.0.1.3680043.9.2095.12.1.151103110937.1...,950b0a71-ca7ffbc4-ac575483-4e2b1f27-3abfc197,4537c253-fbe4854b-38cfb239-ae6d6335-0cb37cdb
2,20120410,12675,1.3.12.2.1107.5.2.30.25658.3000001204100518072...,467436a4-842c2b30-7e59a2fd-828a82eb-43b55f80,429a4e0c-cf55e0ac-4c946d03-8ae810cd-76b18564
3,20111219,11971,1.3.12.2.1107.5.2.30.25658.3000001112190657447...,5b50ccbe-332703b4-0a4244d5-0af9ec24-d69f9914,7eb033b2-daed2e31-63c336f5-64aa9a14-4809977b
4,20130218,14486,1.3.12.2.1107.5.2.30.25658.3000001302180703328...,1d289a1e-40019fd5-460bffc2-59061bc1-54784bf2,ed5dfad9-29150db0-29de3a58-917e883b-7d3fbc6a
...,...,...,...,...,...
1254,20120906,12446,1.3.12.2.1107.5.2.30.25658.3000001209060557370...,2d35d8c3-1b0e268c-9f32ee79-0dd5fe43-37771dd4,b9d266af-05389a81-b1e6c88f-82f241f8-3c69dd28
1255,20120227,12403,1.3.12.2.1107.5.2.30.25658.3000001202270642447...,1b71dc91-042218fe-fb57f52d-6213dc4c-7a2633c2,f5a1b731-27bb735b-71ae9ded-e3bb99e8-6ed13a6d
1256,20180217,AAA172402848,1.3.6.1.4.1.28841.1.10.20.1.9117.17435148.2858...,b9518931-e8a5f3b7-a2832441-d0f29312-3bfee412,2e315654-13d3dd33-85877fd3-27e57801-888f88b5
1257,20121212,14155,1.3.12.2.1107.5.2.30.25658.3000001212120649119...,7a052b9f-257552dd-87fef410-2ffa873f-b635de37,c2d5c99e-e6f4dbaf-22d27ad2-182acad5-e4684113


In [9]:
seq_mapping = pd.read_csv("../meta_data_lesion/sequence_mapping_13056_studies_20240124.csv", sep=';')

In [10]:
merged = pd.merge(metadata_zone, seq_mapping, on='study_orthanc_id', how='left')
merged[merged['adc_tra_id'].isna()]


,study_date,patient_id,StudyInstanceUID,study_orthanc_id,t2w_series_oid,t2w_tra_id,t2w_tra_sd,t2w_sag_id,t2w_sag_sd,t2w_cor_id,...,adc_cor_sd,dwi_tra_id,dwi_tra_sd,dwi_sag_id,dwi_sag_sd,dwi_cor_id,dwi_cor_sd,Manufacturer,PatientID,StudyDate
35,20091217,7415,1.3.12.2.1107.5.2.30.25658.3000000912170706457...,9c562002-475177c7-0ef94c3c-a7df9c56-e4d8db49,180c9607-52b7c93a-a53c44f4-dbadefb5-8cff5058,180c9607-52b7c93a-a53c44f4-dbadefb5-8cff5058,t2_tse_tra_3mm,f3c49ebd-cf94f57f-306aa86b-7d4247c6-ba150097,t2_tse_sag,1b110146-9cf58ec6-1361dc31-469c04f5-71e310fb,...,NaN,089706f4-3db7ecd0-219d2dae-31cd5c86-a5fa1bfd,ep2d_diff_tra,NaN,NaN,77e7b419-afc95d4d-2c6ef16e-4ab19ef0-b95a0c79,ep2d_diff_cor,siemens,7415,20091217.0
201,20110718,11094,1.3.12.2.1107.5.2.30.25658.3000001107180557263...,572b92e3-7e367393-f570574d-fb1d0a7e-f4008671,dd37c7c1-4d03a23f-07858a49-b997ac71-dfbd7cea,dd37c7c1-4d03a23f-07858a49-b997ac71-dfbd7cea,t2_tse_tra,NaN,NaN,e92d23dd-efb952d8-a66ee80b-ea09d1b1-ea155a55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,siemens,11094,20110718.0
573,20100707,8460,1.3.12.2.1107.5.2.30.25658.3000001007070508305...,0cc1ad6a-ce495a56-b7777fbb-5fb4b962-e996e4a2,df343a16-a874928d-8fc47f05-0f304e16-b8b9de54,df343a16-a874928d-8fc47f05-0f304e16-b8b9de54,t2_tse_tra,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,siemens,8460,20100707.0
597,20131115,15837,1.3.12.2.1107.5.2.30.25658.3000001311150556125...,e832ab51-98556add-b46f8a07-302002b1-ad32be38,1948155e-427c2f43-6e30f843-72c00b96-9b7feb43,1948155e-427c2f43-6e30f843-72c00b96-9b7feb43,t2_tse_tra_3mm,NaN,NaN,b910dc10-078033cb-2615204f-282a1c49-c06fa875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,siemens,15837,20131115.0
630,20170505,0041229170,1.2.840.113619.6.95.31.0.3.4.1.1018.13.10831286,22c83766-63cc2b15-ad738fa1-8ca7b520-c6a6e9a9,59238075-174f4a8c-54a613d6-06d028f1-3bdfc344,NaN,NaN,bc4325ce-dc67a80a-2e7b4033-9a490543-402769f2,t2_tse_sagittal_becken_hf,6be28ecd-01b3382e-aa2b54ce-0197fd97-0d727e5a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,siemens,0041229170,20170505.0
703,20140107,14388,1.3.12.2.1107.5.2.30.59050.3000001401070702308...,bc8f5658-101f3504-0da1ee3f-268969d8-21622f9b,de69f92e-d8344838-5d1cc039-75fde2ee-3752205b,de69f92e-d8344838-5d1cc039-75fde2ee-3752205b,t2_tse_tra_320_p2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,siemens,14388,20140107.0


In [12]:
lesion_cases = glob.glob("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset/*/*-tw2_tra.nrrd")
seg_oid_lesion = [i.split("/")[-2] for i in lesion_cases]

In [13]:

lesion_cases    


['/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset/d55838e1-51531d28-e9da1f29-926450d2-1bdcbe85/7d48dbb6-a8896e9c-c43d2ca6-878bf908-0ee68aee-tw2_tra.nrrd',
 '/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset/4c177202-954661e4-997f7da5-dd28eff8-83f62076/30172c4e-a4b83ad0-e99a084e-900398d1-96216e9f-tw2_tra.nrrd',
 '/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset/0ff11760-bd590db0-78fe0467-335a1905-ee6d9520/6a5e264a-c95fcc23-0eb8ac06-481bbe68-ddccc7f4-tw2_tra.nrrd',
 '/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset/cbd6c770-78b7c21f-37ecf9fe-ae375e64-a771c880/4ebbe8c6-0ab2deb2-c61cb0f3-d2bf90a5-2ec05be3-tw2_tra.nrrd',
 '/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset/6f675ad8-8cdb7105-565491c1-4ebbda34-184d991c/6efb2888-376abeef-da254c10-70e93e23-8314a86e-tw2_tra.nrrd',
 '/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset/0eed5870-cef82e9b-5a2d4332-42c911cc-3c3f76da/2fedc341-d17bbf73-a6852c35-1

In [14]:
seg_oid_lesion

['d55838e1-51531d28-e9da1f29-926450d2-1bdcbe85',
 '4c177202-954661e4-997f7da5-dd28eff8-83f62076',
 '0ff11760-bd590db0-78fe0467-335a1905-ee6d9520',
 'cbd6c770-78b7c21f-37ecf9fe-ae375e64-a771c880',
 '6f675ad8-8cdb7105-565491c1-4ebbda34-184d991c',
 '0eed5870-cef82e9b-5a2d4332-42c911cc-3c3f76da',
 'cca9828e-3d2e3d21-17574f10-ebb6a316-229eaffe',
 'd9705fd1-69efe400-35806a29-cad2413f-91e8db35',
 '882fd046-1d9d9072-4668728a-eae7c9bf-ff06a1ee',
 '5b5fc418-824a16bd-4ffcc285-293d45b9-29756660',
 '899ac03b-29baf572-2899b4cd-214bbebb-2d4dd284',
 '765ff694-572a3564-a2746d4b-5bee4065-8140643a',
 'a9d987f2-cdc76f5a-571f5119-6b91b283-794f4b8b',
 '03b292be-f6809bb6-8f2f1df6-f2bc5dda-146db4d4',
 '6c347407-9a12c743-1e4f1ec7-fde76a4a-94f97e34',
 '81080625-ea9f4afe-2e8ce9fa-4ca540fe-e7ff86c2',
 'a6fc8600-6a076bce-7c43e7b5-3fd52001-c5ddbf42',
 '5c600eed-189fbfd2-7c000127-c46113c2-2d05cacb',
 '84943564-dd0741c1-a2c9b810-1db64080-8b18f9f2',
 'b7ab1434-35989fe8-e5cd0893-9f2531ee-1c0a22ec',
 '32a77a5f-bcf27578-

In [18]:

lesion_zone_together = metadata_zone[metadata_zone['study_orthanc_id'].isin(seg_oid_lesion)]

In [16]:
# class_list_big = pd.read_excel('/hdd/drive1/oleksii/Klassenliste für Oleksii.xlsx')
lesion_dataset = pd.read_csv("/home/oleksii/projects/ohif-orthanc-postgres-docker/meta_data_lesion/lesion_dataset_1941_20240126_OB.csv", sep=';')

In [21]:
lesion_zone_together_metadata = lesion_dataset[lesion_dataset['study_orthanc_id'].isin(lesion_zone_together['study_orthanc_id'])]

In [25]:
lesion_zone_together_metadata['csPCa'].value_counts()

False    332
True      62
Name: csPCa, dtype: int64

In [26]:
lesion_zone_together_metadata['case_class'].value_counts()      

CI      163
GS7a     26
GS6      20
GS9      15
GS8      11
GS7b     10
Name: case_class, dtype: int64

In [29]:
lesion_zone_together_metadata['healthy'].value_counts()

False    250
True     144
Name: healthy, dtype: int64

In [28]:
lesion_dataset['healthy'].value_counts()

False    1441
True      500
Name: healthy, dtype: int64

In [30]:
lesion_dataset['case_class'].value_counts()

CI         429
GS7a       329
GS6        232
GS7b       171
GS9        145
GS8         97
GS10        12
HighPIN      3
Name: case_class, dtype: int64

In [37]:

# zonal_cases = glob.glob("/data/oleksii/Prostate-ZONE-Datasets-NRRDS/ALTA-Zone-Dataset/*/Segmentation-label.seg.nrrd")
current_sequence_map = pd.read_csv("../meta_data_lesion/sequence_mapping_13056_studies_20240124.csv", sep=';')
merged = pd.merge(metadata_zone, current_sequence_map, on='study_orthanc_id', how='left')
target_dir_root = "/data/oleksii/Prostate-ZONE-Datasets-NRRDS/ALTA-Zone-Dataset-multimodal-seg/"
os.makedirs(target_dir_root, exist_ok=True)

for i, (_, row) in enumerate(merged.iterrows()):    
    # source_path = os.path.join('/data/oleksii/Prostate-ZONE-Datasets-NRRDS/ALTA-Zone-Dataset', row['t2w_series_oid'], 'Segmentation-label.seg.nrrd')
    source_path = os.path.join('/data/oleksii/Prostate-ZONE-Datasets-NRRDS/ALTA-Zone-Dataset', row['t2w_series_oid'], 'seg.dcm')
    
    if os.path.exists(source_path):
        
        # in case when sequence mapping fails
        if pd.isna(row['t2w_tra_id']):
            t2w_oid = row["t2w_series_oid"]
        else:
            t2w_oid = row['t2w_tra_id']
        
        target_dir = os.path.join(target_dir_root, row['study_orthanc_id'])
        os.makedirs(target_dir, exist_ok=True)
        # shutil.copyfile(source_path, os.path.join(target_dir, 'Segmentation-label.seg.nrrd'))
        shutil.copyfile(source_path, os.path.join(target_dir, 'seg.dcm'))
    


# sftp://141.44.17.135:2200/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-seg/1259274d-4f4a0681-9152ce23-af460ef3-1f9a5935/Segmentation-label-tra-final.seg.nrrd


In [ ]:
"/data/oleksii/Prostate-ZONE-Datasets-NRRDS/ALTA-Zone-Dataset/ffd337d5-5b4e5bbd-57778f99-58d12bcc-f3e48629/Segmentation-label.seg.nrrd"

